In [1]:
import pandas as pd
import numpy as np
#coding=utf-8
import numpy as np
import pandas as pd
from scipy.stats import mode
import xgboost as xgb
from xgboost import XGBRegressor
from xgboost import XGBClassifier
from sklearn.metrics import mean_squared_error,roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from matplotlib import  pyplot
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn import preprocessing  
min_max_scaler = preprocessing.MinMaxScaler()  
from sklearn.model_selection import KFold,StratifiedKFold

In [9]:
fulldata = pd.read_csv("../dealinput/orderCount.csv")
print(fulldata.columns)
print(fulldata.isnull().sum())
fulldata.head()

Index(['id', 'orderInfo_work_time_more_cate', 'orderInfo_week_more_cate',
       'orderInfo_year_more_cate', 'orderInfo_month_more_cate',
       'orderInfo_day_more_cate', 'orderInfo_isoweekday_more_cate',
       'orderInfo_month_stage_more_cate', 'orderInfo_type_pay_more_cate',
       'orderInfo_sts_order_more_cate', 'orderInfo_lossNUm_max',
       'orderInfo_lossNUm_ave', 'orderInfo_work_time_more_proportion',
       'orderInfo_luck', 'orderInfo_name_rec_md5_num',
       'orderInfo_week_more_proportion', 'orderInfo_year_more_proportion',
       'orderInfo_month_more_proportion', 'orderInfo_day_more_proportion',
       'orderInfo_isoweekday_more_proportion',
       'orderInfo_month_stage_more_proportion', 'orderInfo_type_pay_cate_num',
       'orderInfo_type_pay_more_proportion',
       'orderInfo_type_pay_special_proportion',
       'orderInfo_whitePayment_proportion', 'orderInfo_sts_order_cate_num',
       'orderInfo_sts_order_more_proportion', 'orderInfo_phone_cate_num',
       'or

d:\gongju\python3.6 64\anzhuang\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,orderInfo_work_time_more_cate,orderInfo_week_more_cate,orderInfo_year_more_cate,orderInfo_month_more_cate,orderInfo_day_more_cate,orderInfo_isoweekday_more_cate,orderInfo_month_stage_more_cate,orderInfo_type_pay_more_cate,orderInfo_sts_order_more_cate,...,orderInfo_whitePayment_proportion,orderInfo_sts_order_cate_num,orderInfo_sts_order_more_proportion,orderInfo_phone_cate_num,orderInfo_phone_money_max,orderInfo_phone_money_ave,orderInfo_phone_money_proportion,orderInfo_buy_time_interval,orderInfo_buy_money_max,orderInfo_buy_money_ave
0,20160328090001743085,0,37,2015,9,11,5,2,在线支付,完成,...,0.0,1,3.000000,1,0.0,0.0,0.0,8.248731e+06,4888.0,14091.500000
1,20160328160001756263,0,22,2017,5,31,3,3,在线支付,完成,...,0.0,1,1.000000,1,0.0,0.0,0.0,0.000000e+00,99.0,99.000000
2,20160328170001757443,0,22,2017,5,31,3,3,在线支付,完成,...,0.0,1,1.000000,1,0.0,0.0,0.0,0.000000e+00,99.0,99.000000
3,20160328190001761939,0,50,2013,10,12,4,2,货到付款,完成,...,0.0,1,1.057143,1,0.0,0.0,0.0,3.444669e+06,3983.0,368.271429
4,20160329140001784605,0,10,2015,3,12,1,2,在线支付,完成,...,0.0,1,1.500000,2,0.0,0.0,0.0,3.281478e+06,5688.0,1672.086364


In [8]:
fulldata.columns

Index(['id', 'orderInfo_lossNUm_max', 'orderInfo_lossNUm_ave',
       'orderInfo_work_time_more_cate_0', 'orderInfo_work_time_more_cate_1',
       'orderInfo_week_more_cate_1', 'orderInfo_week_more_cate_2',
       'orderInfo_week_more_cate_3', 'orderInfo_week_more_cate_4',
       'orderInfo_week_more_cate_5',
       ...
       'orderInfo_whitePayment_proportion', 'orderInfo_sts_order_cate_num',
       'orderInfo_sts_order_more_proportion', 'orderInfo_phone_cate_num',
       'orderInfo_phone_money_max', 'orderInfo_phone_money_ave',
       'orderInfo_phone_money_proportion', 'orderInfo_buy_time_interval',
       'orderInfo_buy_money_max', 'orderInfo_buy_money_ave'],
      dtype='object', length=191)

In [10]:
'''

'id', 'orderInfo_work_time_more_cate', 'orderInfo_week_more_cate',
       'orderInfo_year_more_cate', 'orderInfo_month_more_cate',
       'orderInfo_day_more_cate', 'orderInfo_isoweekday_more_cate',
       'orderInfo_month_stage_more_cate', 'orderInfo_type_pay_more_cate',
       'orderInfo_sts_order_more_cate', 'orderInfo_lossNUm_max',
       'orderInfo_lossNUm_ave', 'orderInfo_work_time_more_proportion',
       'orderInfo_luck', 'orderInfo_name_rec_md5_num',
       'orderInfo_week_more_proportion', 'orderInfo_year_more_proportion',
       'orderInfo_month_more_proportion', 'orderInfo_day_more_proportion',
       'orderInfo_isoweekday_more_proportion',
       'orderInfo_month_stage_more_proportion', 'orderInfo_type_pay_cate_num',
       'orderInfo_type_pay_more_proportion',
       'orderInfo_type_pay_special_proportion',
       'orderInfo_whitePayment_proportion', 'orderInfo_sts_order_cate_num',
       'orderInfo_sts_order_more_proportion', 'orderInfo_phone_cate_num',
       'orderInfo_phone_money_max', 'orderInfo_phone_money_ave',
       'orderInfo_phone_money_proportion', 'orderInfo_buy_time_interval',
       'orderInfo_buy_money_max', 'orderInfo_buy_money_ave'
'''
cate_feature = ['orderInfo_work_time_more_cate', 'orderInfo_week_more_cate',
       'orderInfo_year_more_cate', 'orderInfo_month_more_cate',
       'orderInfo_day_more_cate', 'orderInfo_isoweekday_more_cate',
       'orderInfo_month_stage_more_cate', 'orderInfo_type_pay_more_cate',
       'orderInfo_sts_order_more_cate']
num_feature = ['orderInfo_lossNUm_max',
       'orderInfo_lossNUm_ave', 'orderInfo_work_time_more_proportion',
       'orderInfo_luck', 'orderInfo_name_rec_md5_num',
       'orderInfo_week_more_proportion', 'orderInfo_year_more_proportion',
       'orderInfo_month_more_proportion', 'orderInfo_day_more_proportion',
       'orderInfo_isoweekday_more_proportion',
       'orderInfo_month_stage_more_proportion', 'orderInfo_type_pay_cate_num',
       'orderInfo_type_pay_more_proportion',
       'orderInfo_type_pay_special_proportion',
       'orderInfo_whitePayment_proportion', 'orderInfo_sts_order_cate_num',
       'orderInfo_sts_order_more_proportion', 'orderInfo_phone_cate_num',
       'orderInfo_phone_money_max', 'orderInfo_phone_money_ave',
       'orderInfo_phone_money_proportion', 'orderInfo_buy_time_interval',
       'orderInfo_buy_money_max', 'orderInfo_buy_money_ave']


In [11]:
from sklearn import preprocessing  
min_max_scaler = preprocessing.StandardScaler()

for i in range(len(cate_feature)):
    tmp = pd.get_dummies(fulldata[cate_feature[i]],prefix = cate_feature[i])
    fulldata = pd.concat([fulldata,tmp],axis = 1) 
    del fulldata[cate_feature[i]]
num = pd.DataFrame(min_max_scaler.fit_transform(fulldata[num_feature]),columns = num_feature)
fulldata.drop(columns = num_feature,inplace = True)
fulldata = pd.concat([fulldata,num],axis = 1) 
# m = fulldata.target.copy()
# del fulldata["target"]
# fulldata.insert(1,"target",m)
fulldata

,id,orderInfo_work_time_more_cate_0,orderInfo_work_time_more_cate_1,orderInfo_week_more_cate_1,orderInfo_week_more_cate_2,orderInfo_week_more_cate_3,orderInfo_week_more_cate_4,orderInfo_week_more_cate_5,orderInfo_week_more_cate_6,orderInfo_week_more_cate_7,...,orderInfo_whitePayment_proportion,orderInfo_sts_order_cate_num,orderInfo_sts_order_more_proportion,orderInfo_phone_cate_num,orderInfo_phone_money_max,orderInfo_phone_money_ave,orderInfo_phone_money_proportion,orderInfo_buy_time_interval,orderInfo_buy_money_max,orderInfo_buy_money_ave
0,20160328090001743085,1,0,0,0,0,0,0,0,0,...,-0.048563,-0.456746,6.786467,-0.290352,-0.316290,-0.324463,-0.298292,2.995212,2.010493,14.103148
1,20160328160001756263,1,0,0,0,0,0,0,0,0,...,-0.048563,-0.456746,-0.045930,-0.290352,-0.316290,-0.324463,-0.298292,-0.373574,-0.576506,-0.324795
2,20160328170001757443,1,0,0,0,0,0,0,0,0,...,-0.048563,-0.456746,-0.045930,-0.290352,-0.316290,-0.324463,-0.298292,-0.373574,-0.576506,-0.324795
3,20160328190001761939,1,0,0,0,0,0,0,0,0,...,-0.048563,-0.456746,0.149281,-0.290352,-0.316290,-0.324463,-0.298292,1.033230,1.521616,-0.047144
4,20160329140001784605,1,0,0,0,0,0,0,0,0,...,-0.048563,-0.456746,1.662169,0.036556,-0.316290,-0.324463,-0.298292,0.966583,2.442650,1.297246
5,20160329160001788684,1,0,0,0,0,0,0,0,0,...,-0.048563,-0.456746,1.418155,-0.290352,-0.316290,-0.324463,-0.298292,0.612923,-0.447399,-0.139635
6,20160329180001790470,1,0,0,0,0,0,0,0,0,...,-0.048563,-0.456746,-0.045930,-0.290352,-0.316290,-0.324463,-0.298292,-0.373574,-0.576506,-0.324795
7,20160329180001791648,1,0,0,0,0,0,0,0,0,...,-0.048563,-0.456746,-0.045930,-0.290352,-0.316290,-0.324463,-0.298292,-0.373574,-0.576506,-0.324795
8,20160329200001794164,0,1,0,0,1,0,0,0,0,...,-0.048563,-0.456746,4.509001,-0.290352,-0.316290,-0.324463,-0.298292,3.585799,-0.478919,0.128073
9,20160330090001805556,1,0,0,0,0,0,0,0,0,...,-0.048563,0.805554,0.978929,0.363465,0.463283,1.024538,0.923126,0.453408,0.449867,-0.049130


In [34]:
len(fulldata.id.unique())

167961

In [5]:
# fulldata.to_csv("../orderOk.csv",index = False)

In [12]:
fulldata = pd.read_csv("../orderOk.csv")
train11 = pd.read_csv("../dealinput/train2ok.csv")
test11 = pd.read_csv("../dealinput/test2ok.csv")
train11.shape,test11.shape

d:\gongju\python3.6 64\anzhuang\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


((120929, 115), (47031, 114))

In [19]:
train11.columns[50:90]

Index(['phoneTop3_184', 'phoneTop3_185', 'phoneTop3_186', 'phoneTop3_187',
       'phoneTop3_188', 'Appl_sbm_tm', 'auth_time_miao', 'credit_score',
       'quota', 'missNumber', 'overdraft', 'timeGap', 'bank_name_more_cate_a',
       'bank_name_more_cate_b', 'bank_name_more_cate_c',
       'bank_name_more_cate_d', 'bank_name_more_cate_e',
       'bank_name_more_cate_f', 'bank_name_more_cate_h',
       'bank_name_more_cate_i', 'bank_name_more_cate_j',
       'bank_name_more_cate_k', 'bank_name_more_cate_l',
       'bank_name_more_cate_n', 'bank_name_more_cate_nk',
       'bank_name_more_cate_o', 'card_type_more_cate_1.0',
       'card_type_more_cate_2.0', 'havaCreditCard_0', 'havaCreditCard_1',
       'phoneSame_0', 'phoneSame_1', 'phone_more_cate_top3_130',
       'phone_more_cate_top3_132', 'phone_more_cate_top3_133',
       'phone_more_cate_top3_134', 'phone_more_cate_top3_135',
       'phone_more_cate_top3_136', 'phone_more_cate_top3_137',
       'phone_more_cate_top3_138'],
      d

In [20]:
fulldata = pd.read_csv("../input/fulldata_order_undeal111.csv", low_memory=False)
uniId = fulldata.id.unique()

In [34]:
len(uniId)
uniId = list(uniId)
kk = pd.DataFrame(data = uniId,columns = "id")
kk

TypeError: Index(...) must be called with a collection of some kind, 'id' was passed

In [26]:
mm = train11[["id","target"]]
# mm.sort_values(by = 'id')
train = mm.merge(kk,how = 'inner',on = 'id')
train

ValueError: can not merge DataFrame with instance of type <class 'pandas.core.series.Series'>

In [33]:
train = pd.concat([train.id,train[ff]],axis = 1)
trainnew = train11.merge(train,how = "inner")

,id,orderInfo_work_time_more_cate_0,orderInfo_work_time_more_cate_1,orderInfo_week_more_cate_8,orderInfo_week_more_cate_13,orderInfo_week_more_cate_31,orderInfo_week_more_cate_33,orderInfo_week_more_cate_34,orderInfo_week_more_cate_45,orderInfo_week_more_cate_48,...,orderInfo_month_stage_more_proportion,orderInfo_type_pay_more_proportion,orderInfo_whitePayment_proportion,orderInfo_sts_order_cate_num,orderInfo_sts_order_more_proportion,orderInfo_phone_cate_num,orderInfo_phone_money_proportion,orderInfo_buy_time_interval,orderInfo_buy_money_max,orderInfo_buy_money_ave
0,445986793985150984,1,0,0,0,0,0,0,0,0,...,-1.152557,-0.672719,-0.048561,0.805476,0.523466,0.036546,1.058802,-0.106110,0.605993,0.129329
1,20160521150003233759,0,1,0,0,0,0,0,0,0,...,-0.637215,-0.100123,-0.048561,-0.456773,-0.045900,-0.290370,-0.298306,-0.341835,-0.544100,-0.341978
2,20160719110005097415,1,0,0,0,0,0,0,0,0,...,0.393469,-0.100123,-0.048561,-0.456773,-0.045900,-0.290370,-0.298306,-0.373596,-0.576513,-0.324793
3,454760633615782152,1,0,0,0,0,0,0,0,0,...,0.393469,-0.100123,-0.048561,-0.456773,-0.045900,-0.290370,5.808679,-0.373596,-0.624601,-0.416585
4,20160512070002956882,1,0,0,0,0,0,0,0,0,...,0.393469,-0.100123,-0.048561,-0.456773,-0.045900,-0.290370,-0.298306,-0.373596,-0.576513,-0.324793
5,20160604220003671017,1,0,0,0,1,0,0,0,0,...,0.393469,-0.100123,-0.048561,-0.456773,-0.045900,-0.290370,5.808679,-0.373596,-0.619297,-0.406459
6,20160530200003508901,1,0,0,0,0,0,0,0,0,...,0.393469,-0.100123,-0.048561,-0.456773,-0.045900,-0.290370,-0.298306,-0.373596,-0.576513,-0.324793
7,20160716210005025748,1,0,0,0,0,0,0,0,0,...,0.135798,-1.817910,-0.048561,0.805476,-0.899950,-0.290370,-0.298306,0.967972,-0.360431,-0.300406
8,20160628080004407255,1,0,0,0,0,0,0,0,0,...,-1.216975,-0.601144,-0.048561,0.805476,-0.686438,1.671126,1.228440,0.414326,0.714034,-0.031379
9,20160729130005429977,1,0,0,0,0,0,0,0,0,...,-1.514393,0.082620,-0.048561,0.805476,-1.535945,-0.290370,3.144994,-0.218905,-0.575983,-0.388729


In [4]:
train.isnull().sum()

id                                       0
target                                   0
orderInfo_work_time_more_cate_0          0
orderInfo_work_time_more_cate_1          0
orderInfo_week_more_cate_1               0
orderInfo_week_more_cate_2               0
orderInfo_week_more_cate_3               0
orderInfo_week_more_cate_4               0
orderInfo_week_more_cate_5               0
orderInfo_week_more_cate_6               0
orderInfo_week_more_cate_7               0
orderInfo_week_more_cate_8               0
orderInfo_week_more_cate_9               0
orderInfo_week_more_cate_10              0
orderInfo_week_more_cate_11              0
orderInfo_week_more_cate_12              0
orderInfo_week_more_cate_13              0
orderInfo_week_more_cate_14              0
orderInfo_week_more_cate_15              0
orderInfo_week_more_cate_16              0
orderInfo_week_more_cate_17              0
orderInfo_week_more_cate_18              0
orderInfo_week_more_cate_19              0
orderInfo_w

In [30]:
# trainx = train.iloc[:,2:]
trainx = train[ff]
trainy = train.iloc[:,1]
print("trainx.shape",trainx.shape)
print("trainy.shape",trainy.shape)
from sklearn.cross_validation import train_test_split
x_train,x_valid,y_train,y_valid = train_test_split(trainx,trainy,test_size = 0.3,random_state = 2)  
print("x_train.shape",x_train.shape)
print("x_valid.shape",x_valid.shape)
print("y_train.shape",y_train.shape)
print("y_valid.shape",y_valid.shape)

trainx.shape (19080, 49)
trainy.shape (19080,)
x_train.shape (13356, 49)
x_valid.shape (5724, 49)
y_train.shape (13356,)
y_valid.shape (5724,)


In [31]:
skf = StratifiedKFold(n_splits=5)
xgb1 = XGBClassifier(
#         booster = 'dart',
        learning_rate =0.1,
        n_estimators=5000,
        max_depth=5,
        min_child_weight=5,
        gamma=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        objective= 'binary:logistic',
        nthread=2,
        scale_pos_weight=1,
        StratifiedKFold = True)          
xgb_param = xgb1.get_xgb_params()
#构建稀疏矩阵，运行更快
xgtrain = xgb.DMatrix(x_train, label = y_train)
mm = xgb.cv(xgb_param, xgtrain, num_boost_round=1000, nfold=5,
    metrics='auc', early_stopping_rounds=50)
mm.iloc[-3:,:]  
#0.594478
# 0.013305
# 0.630512
# 0.005787

,test-auc-mean,test-auc-std,train-auc-mean,train-auc-std
14,0.597438,0.009367,0.665475,0.005257
15,0.597403,0.010503,0.666793,0.003587
16,0.601397,0.013649,0.671949,0.003468


In [28]:
importance = 0
for i in range(5):
    xgb1 = XGBClassifier(
    #         booster = 'dart',
            learning_rate =0.1,
            n_estimators=mm.shape[0],
            max_depth=5,
            min_child_weight=5,
            gamma=0.1,
            subsample=0.8,
            colsample_bytree=0.8,
            objective= 'binary:logistic',
            nthread=3,
            scale_pos_weight=1,
            StratifiedKFold = True,          #0.866124	0.008005	0.965337	0.002286
            seed=1)
    xgb1.fit(x_train,y_train)
    prexgb = xgb1.predict_proba(x_valid)[:,1]
    auc = roc_auc_score(y_valid,prexgb)
    print("xgb1 auc",auc)
    importance += xgb1.feature_importances_
importance = importance/5

xgb1 auc 0.5913493179370588
xgb1 auc 0.5913493179370588
xgb1 auc 0.5913493179370588
xgb1 auc 0.5913493179370588
xgb1 auc 0.5913493179370588


In [29]:
print(trainx.columns,importance)
number = 0
ff = []
for i in range(len(importance)):
    if importance[i] >0.001:
        ff.append(trainx.columns[i])
        number += 1
print(number)
ff,len(ff)

Index(['orderInfo_work_time_more_cate_0', 'orderInfo_work_time_more_cate_1',
       'orderInfo_week_more_cate_1', 'orderInfo_week_more_cate_2',
       'orderInfo_week_more_cate_3', 'orderInfo_week_more_cate_4',
       'orderInfo_week_more_cate_5', 'orderInfo_week_more_cate_6',
       'orderInfo_week_more_cate_7', 'orderInfo_week_more_cate_8',
       ...
       'orderInfo_whitePayment_proportion', 'orderInfo_sts_order_cate_num',
       'orderInfo_sts_order_more_proportion', 'orderInfo_phone_cate_num',
       'orderInfo_phone_money_max', 'orderInfo_phone_money_ave',
       'orderInfo_phone_money_proportion', 'orderInfo_buy_time_interval',
       'orderInfo_buy_money_max', 'orderInfo_buy_money_ave'],
      dtype='object', length=188) [0.00540541 0.00540541 0.         0.         0.         0.
 0.         0.         0.         0.00540541 0.         0.
 0.         0.         0.01621622 0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         

(['orderInfo_work_time_more_cate_0',
  'orderInfo_work_time_more_cate_1',
  'orderInfo_week_more_cate_8',
  'orderInfo_week_more_cate_13',
  'orderInfo_week_more_cate_31',
  'orderInfo_week_more_cate_33',
  'orderInfo_week_more_cate_34',
  'orderInfo_week_more_cate_45',
  'orderInfo_week_more_cate_48',
  'orderInfo_week_more_cate_50',
  'orderInfo_year_more_cate_2015',
  'orderInfo_month_more_cate_3',
  'orderInfo_month_more_cate_4',
  'orderInfo_month_more_cate_7',
  'orderInfo_month_more_cate_8',
  'orderInfo_month_more_cate_10',
  'orderInfo_month_more_cate_11',
  'orderInfo_day_more_cate_5',
  'orderInfo_day_more_cate_7',
  'orderInfo_day_more_cate_16',
  'orderInfo_day_more_cate_19',
  'orderInfo_day_more_cate_24',
  'orderInfo_day_more_cate_29',
  'orderInfo_day_more_cate_30',
  'orderInfo_isoweekday_more_cate_2',
  'orderInfo_isoweekday_more_cate_3',
  'orderInfo_isoweekday_more_cate_5',
  'orderInfo_month_stage_more_cate_3',
  'orderInfo_type_pay_more_cate_货到付款',
  'orderInfo_s